**Universidade Federal do Rio Grande do Sul**  
Departamento de Engenharia Civil (DECIV)  
ENG01007 - Análise Estrutural por Computador  
*prof. Felipe Schaedler de Almeida*

---


# Exemplo: Implementação de uma função para cálculo de matriz de rigidez global de um elemento de treliça plana




In [ ]:
#Importando módulos necessários para esse notebook
from math import *
import numpy as np

As forças e deslocamentos do elemento de treliça plana são relacionadas pela equação de rigidez

\begin{equation*}
\boldsymbol{q} = \boldsymbol{K} \boldsymbol{d}
\end{equation*}

Para um elemento com comrimento($L$) e inclinação ($\phi$) em relação ao eixo $x+$,

<img src="T2D_d+q_global.png" alt="elemento_de_trelica" style="width:250px"/>


a equação de rigidez pode ser expressa por:

$\newcommand{\scf}{\sin \! \phi \cos \! \phi}$
$\newcommand{\ssf}{\sin^2 \! \phi}$
$\newcommand{\ccf}{\cos^2 \! \phi}$

\begin{equation*}
\begin{Bmatrix} f_{xi} \\ f_{zi} \\ f_{xj} \\ f_{zj} \end{Bmatrix} = \frac{EA}{L}
  \left[ \begin{array}{cc>{\columncolor{yellow}}cc}
  \ccf &   \scf & - \ccf & - \scf \\
  \scf &   \ssf & - \scf & - \ssf \\
- \ccf & - \scf &   \ccf &   \scf \\
- \scf & - \ssf &   \scf &   \ssf
 \end{array} \right]
\begin{Bmatrix} u_{xi} \\ u_{zi} \\ u_{xj} \\ u_{zj} \end{Bmatrix}
\end{equation*}


onde $A$ é a área da seção transversal e $E$ é o módulo de elasticidade do material.


In [ ]:

def T2D_K_global_phi(E,A,L,phi):
    
    """
    Calcula da matriz de rigidez global de uma barra de treliça plana (2D)
    em função do ângulo de inclinação da barra em relação ao eixo x (phi em radianos)
    
    Parameters
    ----------
    E : float
        Módulo de elasticidade do material.
    A: float
        Área da seção transversal da barra
    L: float
        Comprimento da barra
    phi: float
        Ângulo entre o eixo x+ e o eixo da barra, em radianos, tomado positivo no sentido anti-horário
    """
    
    r = E*A/L                 #rigidez axial da seção
    sc = r*sin(phi)*cos(phi)  #sen * cos * EA/L
    c2 = r*cos(phi)**2        #cos * cos * EA/L 
    s2 = r*sin(phi)**2        #sen * sen * EA/L 


    K = np.zeros((4,4)) #matriz de rigidez 4x4 (iniciando com zeros)
    
   
    K[0,  : ] = [c2, sc, -c2, -sc] #1º linha
    K[1,  : ] = [sc, s2, -sc, -s2] #2º linha
    K[2,  : ] = - K[0,  : ]        #3º linha igual 1º linha vezes (-1)
    K[3,  : ] = - K[1,  : ]        #4º linha igual 2º linha vezes (-1)
    
    return K

In [ ]:
#mostrando a docstring da função implementada
T2D_K_global_phi?
#np?

## Teste do uso da função T2D_K_global_phi:

Barras ab, ac e bc do exemplo 3.4 (pg.42) do livro Matrix Structural Analysis,  2º ed. (W. MCGuire, R.H. Gallagher, R.D. Ziemian, 2014)

<img src="MSA_fig_ex3.4.png" alt="elemento_de_trelica" style="width:250px"/>


In [ ]:
#unidades : L=[mm], F=[N]

#propriedades do material:
E = 200.0e3 #[MPa= N/mm²] 

#Barra ab:
A_ab = 10.0e3 #mm²
L_ab = 5.0e3 #mm
ang_ab = 0 #em graus
phi_ab = pi * ang_ab/180.0 #em readianos ; phi_ab = radians(ang_ab)
K_ab = T2D_K_global_phi(E,A_ab,L_ab,phi_ab)
print('\n K_ab / (EA/L)= ')
print(K_ab / (E*A_ab/L_ab))

#Barra ac:
A_ac = 15.0e3 #mm²
ang_ac = 60 #em graus
phi_ac = pi * ang_ac/180.0 #em readianos
L_ac = 5.0e3 /cos(phi_ac) #mm
K_ac = T2D_K_global_phi(E,A_ac,L_ac,phi_ac)
print('\n K_ac / (EA/L)= ')
print(K_ac / (E*A_ac/L_ac))

#Barra bc:
A_bc = 15.0e3 #mm²
ang_bc = 90 #em graus
phi_bc = pi * ang_bc/180.0 #em readianos
L_bc = 5.0e3 * tan(radians(60)) #mm
K_bc = T2D_K_global_phi(E,A_bc,L_bc,phi_bc)
print('\n K_bc / (EA/L)= ')
print(K_bc / (E*A_bc/L_bc))


In [ ]:
print('\n EA/L = ',E*A_ab/L_ab)

# Implementação alternativa da subrotina usando objetos para descrever as propriedades da seção transversal:

Definido uma classe para as propriedades dos materiais:

In [ ]:
class Material():
    '''
    Material elástico linear para análise estrutural por AECPy
    
    Atributes
    ----------
    E: float
        Módulo de Young
    G: float
        Módulo de cisalhamento
    cp: float
        Coeficiente de Poisson
    pe: float
        Peso específico
    cdt: float
        Coeficiente de dilatação térmica
    nome: str
        Nome do material
    '''

    def __init__(self,E,cp,pe=0,cdt=0,nome=""):
        '''
        Inicialização de uma instância da classe Material

        Parametres
        ----------
        E: float
            Módulo de Young
        cp: float
            Coeficiente de Poisson
        pe: float (optional)
            Peso específico (default 0)
        cdt: float (optional)
            Coeficiente de dilatação térmica (default 0)
        nome: str (optional)
            Nome do material (default "")

        Raises
        ------
        ValueError
            Se qualquer propriedade do material for negativa 
        '''

        if any([ prop<0 for prop in [E,cp,pe,cdt]]):
                raise ValueError('As propriedades do material não devem ser negativas')

        self.E = E      #módulo de Young (de elasticidade)
        self.cp = cp    #coeficiente de poisson
        self.G = E/(2*(1+cp)) #módulo de cisalhamento (de elasticidade transversal)
        self.pe = pe    #peso específico
        self.cdt = cdt  #Coefciente de dilatação térmica
        self.nome = nome #nome do material

Definido uma classe para as propriedades da seção transversal:

In [ ]:
class Secao():

    '''
    Seção transversal para um elemento de barra prismático
    usado na análise estrutural por AECPy

    Na denominação dos atributos, são considerados:
    eixo local 2 - eixo vertical
    eixo local 3 - eixo horizonal
    Os eixos 2 e 3 são eixos principais centrais de inércia da seção
    
    Atributes
    ----------
    mat: Material
        Material que forma o elemento estrutural
    A: float
        Área da seção transveral
    I2: float
        Momento de inércia em relação ao eixo local 2
    I3: float
        Momento de inércia em relação ao eixo local 3
    J: float
        Constante de torção pura (constante de St. Venant)
    AE: float
        Rigidez axial da seção
    EI2: float
        Rigidez da seção à flexão em torno do eixo local 2
    EI3: float
        Rigidez da seção à flexão em torno do eixo local 3 
    GJ: float
        Rigidez à torção pura da seção
    nome: str
        Nome do material
    peso_unitario
    r2
    r3
    
    '''

    def __init__(self,mat,A,I3=0,I2=0,J=0,nome=''):
        '''
        mat: Material
            Material que forma o elemento estrutural
        A: float
            Área da seção transveral
        I2: float (optional)
            Momento de inércia em relação ao eixo local 2. (deault é 0, aplicável à análise de treliças)
        I3: float (optional)
            Momento de inércia em relação ao eixo local 3 (default é 0, não aplicável para análise de pórticos espaciais)
        J: float (optional)
            Constante de torção pura (constante de St. Venant) (default é 0, não aplicável à análise de pórticos espaciais)
        nome: str (optional)
            Nome da seção transveral (default "")

        Raises
        ------
        ValueError
            Se qualquer propriedade geométrica for negativa
        '''
        
        if not isinstance(mat,Material):
            raise TypeError("mat deve ser do tipo Material")
        if any([ prop<0 for prop in [A,I2,I3,J]]):
                raise ValueError('As propriedades geométricas não devem ser negativas')

        self.mat = mat   #material 
        self.A = A      #Área da seção
        self.I2 = I2    #Momento de inérica da seção em relação ao eixo local 2
        self.I3 = I3    #Momento de inérica da seção em relação ao eixo local 3
        self.J = J      #Constante de torção pura (St. Venant)            
        self.nome = nome

        self.EA = mat.E*self.A     #Rigidez da seção ao alongamento/encurtamento axial
        self.EI2 = mat.E*self.I2   #Ridigez da seção à flexão em torno do eixo local 2 
        self.EI3 = mat.E*self.I3   #Ridigez da seção à flexão em torno do eixo local 3
        self.GJ = mat.G*self.J     #Rigidez da seção à torção pura (St. Venant)


    @property
    def peso_unitario(self):
        '''Peso por unidade de comprimento (da barra)'''
        return self.mat.pe * self.A
    
    @property
    def r2(self):
        '''Raio de giração em relação ao eixo 2'''
        return math.sqrt(self.I2/self.A)
    
    @property
    def r3(self):
        '''Raio de giração em relação ao eixo 3'''
        return math.sqrt(self.I3/self.A)


Definindo a função T2D_K_global_phi usando objetos do tipo Secao como argumento:

In [ ]:
def T2D_K_global_phi2(S,L,phi):
    
    """
    Calcula da matriz de rigidez global de uma barra de treliça plana (2D)
    em função do ângulo %debug inclinação da barra em relação ao eixo x (phi em radianos)
    
    Parameters
    ----------
    S: Secao
        Propriedades da seção transversal da barra
    L: float
        Comprimento da barra
    phi: float
        Ângulo entre o eixo x+ e o eixo da barra, em radianos, tomado positivo no sentido anti-horário
    """
    
    r = S.EA/L                #rigidez axial da seção
    sc = r*sin(phi)*cos(phi)  #sen * cos * EA/L
    c2 = r*cos(phi)**2        #cos * cos * EA/L 
    s2 = r*sin(phi)**2        #sen * sen * EA/L 

    K = np.zeros((4,4)) #matriz de rigidez 4x4 (iniciando com zeros)
    
    K[0,  : ] = [c2, sc, -c2, -sc] #1º linha
    K[1,  : ] = [sc, s2, -sc, -s2] #2º linha
    K[2,  : ] = - K[0,  : ]        #3º linha igual 1º linha vezes (-1)
    K[3,  : ] = - K[1,  : ]        #4º linha igual 2º linha vezes (-1)
    
    return K

Criando uma função para o cálculo do comprimento (L) e ângulo de inclinação da barra (phi) no plano com base nas coordenadas dos nós:

In [ ]:
def T2D_L_phi(rI, rJ):
    """
    Calcula o comprimento (L) e o ângulo de inclinação (phi) em relação ao eixo 'x+'
    para um elemento em um problema plano (2D)
    
    Parameters
    ----------
    rI, rJ: numpy.ndarray
        Arrays com as coordenadas dos nós inicial (I) e final (J) do elemento
    """    

    #vetor posição relativa entre os nós da barra
    rIJ = rJ - rI
    #comprimento da barra
    L = np.linalg.norm(rIJ)
    #ângulo de inclinação da barra (em rad) em relação ao eixo x+
    phi = atan2(rIJ[1],rIJ[0])
    
    return L, phi

Teste do uso da função T2D_K_global_phi2 e T2D_L_phi:

Barras ab, ac e bc do exemplo 3.4 do livro Matrix Structural Analysis, 2º ed. (W. MCGuire, R.H. Gallagher, R.D. Ziemian, 2014)

In [ ]:
#unidades : L=[mm], F=[N]

#propriedades do material:
#E = 200.0e3 [MPa= N/mm²] , coef. Poisson = 0.3
propmat = Material(200.0e3, 0.3)

#seções sec_1 para a barra ab e sec_2 para as barras ac e bc:
#A_1 = 10.0e3 #mm² -- demais propriedades: adotando valor qualquer
sec_1 = Secao(propmat,10.0e3,1,1,1)

#A_2 = 15.0e3 #mm² -- demais propriedades: adotando valor qualquer
sec_2 = Secao(propmat,15.0e3,1,1,1)


#coordenadas dos nós:
ra = np.array([0     , 0])
rb = np.array([5.0e3 , 0])
rc = np.array([5.0e3, 5.0e3/tan(pi*2/3)])

#Barra ab:
L_ab, phi_ac = T2D_L_phi(ra,rb)
K_ab = T2D_K_global_phi2(sec_1,L_ab,phi_ab)
print('\n K_ab / (EA/L)= ')
print(K_ab / (sec_1.EA/L_ab))

#Barra ac:
L_ac, phi_ac = T2D_L_phi(ra,rc)
K_ac = T2D_K_global_phi2(sec_2,L_ac,phi_ac)
print('\n K_ac / (EA/L)= ')
print(K_ac / (sec_2.EA/L_ac))

#Barra bc:
L_bc, phi_bc = T2D_L_phi(rb,rc)
K_bc = T2D_K_global_phi2(sec_2,L_bc,phi_bc)
print('\n K_bc / (EA/L)= ')
print(K_bc / (sec_2.EA/L_bc))

